# DSC 275/475: Time Series Analysis and Forecasting (Fall 2019)
## Project 2.2.2 – Sequence Classification with Recurrent Neural Networks
### Chunlei Zhou

In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import unicodedata
import string
import torch
import torch.nn as nn
import random
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from torch import LongTensor
from torch.nn import Embedding
from sklearn.metrics import accuracy_score
from torch.autograd import Variable

In [2]:
class RNN_Batch(nn.Module):
    def __init__(self):
        super(RNN_Batch, self).__init__()
        self.rnn = nn.RNN(
            input_size=n_letters,
            hidden_size=128, 
            num_layers=1,  
            batch_first=True,
        )
        self.out = nn.Linear(128, n_categories)

    def forward(self, x):
        r_out, h = self.rnn(x, None)
        out = self.out(r_out[:,-1,:])
        return out

class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__()

        self.rnn = nn.LSTM(
            input_size = n_letters,
            hidden_size = 128,
            num_layers = 1,
            batch_first = True,
        )
        self.out = nn.Linear(128, n_categories)

    def forward(self, x):
        r_out, (h_n, h_c) = self.rnn(x, None)   
        out = self.out(r_out[:, -1, :])
        return out

In [3]:
def findFiles(path):
    return glob.glob(path)

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters)

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles('data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

# Construct a Data Frame
def data_frame(category_lines):
    data_frame = [[],[]]
    for key in all_categories:
        for value in category_lines[key]:
            data_frame[0].append(value)
            data_frame[1].append(key)
    return data_frame

In [4]:
DF = data_frame(category_lines)
total_samples = len(DF[0])
randomize_order = np.arange(0, total_samples)
learning_rate = 0.005

# 1 Batch Training of Data
## 1.1 Leverage the RNN for batch training

In [5]:
# Padding
feature = sorted(set(all_letters))
vectorized_seqs = [[feature.index(tok) for tok in seq]for seq in DF[0]]
print(DF[0][:5])
print(vectorized_seqs[:5])
embed = Embedding(len(feature), n_letters)
seq_lengths = LongTensor(list(map(len, vectorized_seqs)))
seq_tensor = Variable(torch.zeros((len(vectorized_seqs), seq_lengths.max()))).long()
for idx, (seq, seqlen) in enumerate(zip(vectorized_seqs, seq_lengths)):
    seq_tensor[idx, :seqlen] = LongTensor(seq)
print(seq_tensor)
seq_tensor.shape

['Abl', 'Adsit', 'Ajdrna', 'Alt', 'Antonowitsch']
[[5, 32, 42], [5, 34, 49, 39, 50], [5, 40, 34, 48, 44, 31], [5, 42, 50], [5, 44, 50, 45, 44, 45, 53, 39, 50, 49, 33, 38]]
tensor([[ 5, 32, 42,  ...,  0,  0,  0],
        [ 5, 34, 49,  ...,  0,  0,  0],
        [ 5, 40, 34,  ...,  0,  0,  0],
        ...,
        [30, 39, 35,  ...,  0,  0,  0],
        [30, 39, 35,  ...,  0,  0,  0],
        [30, 51, 48,  ...,  0,  0,  0]])


torch.Size([20074, 19])

In [6]:
embedded_seq_tensor = embed(seq_tensor)
print(embedded_seq_tensor)
embedded_seq_tensor.shape

tensor([[[ 1.5352,  0.5698, -0.3237,  ..., -2.4026,  1.1965,  0.7177],
         [-0.4861,  0.0046, -0.2793,  ..., -0.1003, -2.1389,  0.4859],
         [ 1.2304,  0.7228,  1.5585,  ...,  1.1644,  1.1901,  1.9890],
         ...,
         [-0.1933, -0.7656,  1.7654,  ...,  0.5199,  1.9145, -0.4654],
         [-0.1933, -0.7656,  1.7654,  ...,  0.5199,  1.9145, -0.4654],
         [-0.1933, -0.7656,  1.7654,  ...,  0.5199,  1.9145, -0.4654]],

        [[ 1.5352,  0.5698, -0.3237,  ..., -2.4026,  1.1965,  0.7177],
         [-1.3778,  0.2630,  1.3235,  ...,  3.4291,  0.5275,  0.1177],
         [-1.4819, -0.6776, -0.5344,  ..., -0.7741, -0.0990,  1.1569],
         ...,
         [-0.1933, -0.7656,  1.7654,  ...,  0.5199,  1.9145, -0.4654],
         [-0.1933, -0.7656,  1.7654,  ...,  0.5199,  1.9145, -0.4654],
         [-0.1933, -0.7656,  1.7654,  ...,  0.5199,  1.9145, -0.4654]],

        [[ 1.5352,  0.5698, -0.3237,  ..., -2.4026,  1.1965,  0.7177],
         [-0.3903, -0.7238,  0.8776,  ...,  2

torch.Size([20074, 19, 57])

In [7]:
target = sorted(set(all_categories))
vectorized_y = [target.index(tok) for tok in DF[1]]
print(DF[1][:5])
print(vectorized_y[:5])
target_tensor = torch.tensor(vectorized_y,dtype = torch.long)
target_tensor.shape

['Czech', 'Czech', 'Czech', 'Czech', 'Czech']
[2, 2, 2, 2, 2]


torch.Size([20074])

In [8]:
n_hidden = 128
batch_size = total_samples
print('======= 1.1 Accuracy Report =======')
print('Batch Size:', batch_size)

======= 1.1 Accuracy Report =======
Batch Size: 20074


In [9]:
n_epoch = 5
rnn = RNN_Batch()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
loss_func = nn.CrossEntropyLoss()

In [10]:
batch = [embedded_seq_tensor,target_tensor]

In [11]:
x = batch[0]
y = batch[1]
n_epoch = 5
for epoch in range(n_epoch): 
    optimizer.zero_grad()
    output = rnn(x)
    loss = loss_func(output, y) 
    loss.backward(retain_graph=True)
    optimizer.step()
    pred = torch.max(output, 1)[1]
    accuracy = accuracy_score(y, pred)
    print("Epoch: ", epoch, "| train loss: %.4f" % loss.item(), '| test accuracy: %f' % accuracy)

Epoch:  0 | train loss: 2.8147 | test accuracy: 0.036067
Epoch:  1 | train loss: 2.0949 | test accuracy: 0.468666
Epoch:  2 | train loss: 1.9233 | test accuracy: 0.468666
Epoch:  3 | train loss: 1.8931 | test accuracy: 0.468666
Epoch:  4 | train loss: 1.8866 | test accuracy: 0.468666


## 1.2 Arbitrary mini-batch sizes

In [12]:
print('======= 1.2 Accuracy Report =======')
batch_sizes = [1000, 2000, 5000]
N = 20000
for batch_size in batch_sizes:
    print('Batch Size =', batch_size)
    rnn = RNN_Batch()
    optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
    loss_func = nn.CrossEntropyLoss()
    n_epoch = int(N/batch_size)
    for epoch in range(n_epoch):
        samples = random.sample(list(randomize_order), 20000)
        sample_index = [samples[i * batch_size:(i + 1) * batch_size] for i in range((len(samples) + batch_size - 1) // batch_size )]
        batch_tensors = []
        target_tensors = []
        sample = []
        target_output = []
        for index in sample_index:
            for i in index:
                sample.append(DF[0][i])
                target_output.append(DF[1][i])
            vectorized_seqs = [[feature.index(tok) for tok in seq]for seq in sample]
            seq_lengths = LongTensor(list(map(len, vectorized_seqs)))
            seq_tensor = Variable(torch.zeros((len(vectorized_seqs), seq_lengths.max()))).long()
            for idx, (seq, seqlen) in enumerate(zip(vectorized_seqs, seq_lengths)):
                seq_tensor[idx, :seqlen] = LongTensor(seq)
            embedded_seq_tensor = embed(seq_tensor)
            batch_tensors.append(embedded_seq_tensor)
            vectorized_y = [target.index(tok) for tok in target_output]
            target_tensor = torch.tensor(vectorized_y,dtype = torch.long)
            target_tensors.append(target_tensor)  
        for j in range(n_epoch):
            batch = [batch_tensors[j],target_tensors[j]]
            x = batch[0]
            y = batch[1]
            optimizer.zero_grad()
            output = rnn(x)
            loss = loss_func(output, y) 
            loss.backward(retain_graph=True)
            optimizer.step()
        test_output = rnn(embedded_seq_tensor)
        pred = torch.max(test_output, 1)[1]
        accuracy = accuracy_score(y, pred)
        print("Epoch: ", epoch, "| train loss: %.4f" % loss.item(), '| test accuracy: %f' % accuracy)

======= 1.2 Accuracy Report =======
Batch Size = 1000
Epoch:  0 | train loss: 1.8513 | test accuracy: 0.469400
Epoch:  1 | train loss: 1.8439 | test accuracy: 0.468950
Epoch:  2 | train loss: 1.8430 | test accuracy: 0.469100
Epoch:  3 | train loss: 1.8445 | test accuracy: 0.468350
Epoch:  4 | train loss: 1.8435 | test accuracy: 0.468750
Epoch:  5 | train loss: 1.8449 | test accuracy: 0.468500
Epoch:  6 | train loss: 1.8444 | test accuracy: 0.468650
Epoch:  7 | train loss: 1.8445 | test accuracy: 0.468450
Epoch:  8 | train loss: 1.8439 | test accuracy: 0.468800
Epoch:  9 | train loss: 1.8433 | test accuracy: 0.468650
Epoch:  10 | train loss: 1.8442 | test accuracy: 0.468400
Epoch:  11 | train loss: 1.8442 | test accuracy: 0.468500
Epoch:  12 | train loss: 1.8430 | test accuracy: 0.468750
Epoch:  13 | train loss: 1.8412 | test accuracy: 0.468750
Epoch:  14 | train loss: 1.8480 | test accuracy: 0.468700
Epoch:  15 | train loss: 1.8450 | test accuracy: 0.468550
Epoch:  16 | train loss: 1.8

# 2. Model cross-validation
## 2.1 Five-Fold Cross-Validation

In [13]:
feature = sorted(set(all_letters))
vectorized_seqs = [[feature.index(tok) for tok in seq]for seq in DF[0]]
print(DF[0][:5])
print(vectorized_seqs[:5])
embed = Embedding(len(feature), n_letters)
seq_lengths = LongTensor(list(map(len, vectorized_seqs)))
seq_tensor = Variable(torch.zeros((len(vectorized_seqs), seq_lengths.max()))).long()
for idx, (seq, seqlen) in enumerate(zip(vectorized_seqs, seq_lengths)):
    seq_tensor[idx, :seqlen] = LongTensor(seq)
print(seq_tensor)
embedded_seq_tensor = embed(seq_tensor)
print(embedded_seq_tensor)
print(embedded_seq_tensor.shape)
target = sorted(set(all_categories))
vectorized_y = [target.index(tok) for tok in DF[1]]
print(DF[1][:5])
print(vectorized_y[:5])
target_tensor = torch.tensor(vectorized_y,dtype = torch.long)
print(target_tensor.shape)

['Abl', 'Adsit', 'Ajdrna', 'Alt', 'Antonowitsch']
[[5, 32, 42], [5, 34, 49, 39, 50], [5, 40, 34, 48, 44, 31], [5, 42, 50], [5, 44, 50, 45, 44, 45, 53, 39, 50, 49, 33, 38]]
tensor([[ 5, 32, 42,  ...,  0,  0,  0],
        [ 5, 34, 49,  ...,  0,  0,  0],
        [ 5, 40, 34,  ...,  0,  0,  0],
        ...,
        [30, 39, 35,  ...,  0,  0,  0],
        [30, 39, 35,  ...,  0,  0,  0],
        [30, 51, 48,  ...,  0,  0,  0]])
tensor([[[-0.7040, -0.2269,  1.2852,  ...,  1.3787, -0.7968,  1.3350],
         [ 1.0751,  1.3456,  0.2650,  ...,  0.0063,  0.3570,  0.9540],
         [ 0.8692,  0.9518, -2.3914,  ..., -0.9330,  0.1102, -0.8087],
         ...,
         [-0.1318,  0.7543,  0.5666,  ...,  0.7242, -1.1255, -0.8621],
         [-0.1318,  0.7543,  0.5666,  ...,  0.7242, -1.1255, -0.8621],
         [-0.1318,  0.7543,  0.5666,  ...,  0.7242, -1.1255, -0.8621]],

        [[-0.7040, -0.2269,  1.2852,  ...,  1.3787, -0.7968,  1.3350],
         [ 2.3416, -0.7289,  0.7296,  ..., -0.3200, -0.3559, 

###  (a) Accuracy Report After Five Epochs

In [ ]:
print('======= 2.1(a) Accuracy Report After Five Epochs =======')
kfold_cv = KFold(n_splits=5, random_state=None, shuffle=True)
rnn = RNN_Batch()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
loss_func = nn.CrossEntropyLoss()
n_epoch = 5 
for epoch in range(n_epoch):
    accuracies_test = []
    accuracies_train = []
    for train_index, test_index in kfold_cv.split(np.arange(0, total_samples)):
        X_train, X_test, y_train, y_test = embedded_seq_tensor[train_index], embedded_seq_tensor[test_index], target_tensor[train_index], target_tensor[test_index]
        optimizer.zero_grad()
        output = rnn(X_train)
        loss = loss_func(output, y_train) 
        loss.backward(retain_graph=True)
        optimizer.step()
        test_output = rnn(X_test)
        train_output = rnn(X_train)
        pred_test = torch.max(test_output, 1)[1]
        pred_train = torch.max(train_output, 1)[1]
        accuracies_test.append(accuracy_score(y_test, pred_test))
        accuracies_train.append(accuracy_score(y_train, pred_train))
    print("Epoch: ", epoch, "| Train Loss: %.4f" % loss.item(), '| Test accuracy: %f' % np.mean(accuracies_test))
    print("Epoch: ", epoch, "| Train Loss: %.4f" % loss.item(), '| Train accuracy: %f' % np.mean(accuracies_train))

### (b) Accuracy Report After Chosen Epochs

In [16]:
# Padding
feature = sorted(set(all_letters))
vectorized_seqs = [[feature.index(tok) for tok in seq]for seq in DF[0]]
print(DF[0][:5])
print(vectorized_seqs[:5])
embed = Embedding(len(feature), n_letters)
seq_lengths = LongTensor(list(map(len, vectorized_seqs)))
print(seq_lengths)
seq_tensor = Variable(torch.zeros((len(vectorized_seqs), 9))).long()
print(seq_tensor)
seq_tensor.shape
for idx, (seq, seqlen) in enumerate(zip(vectorized_seqs, seq_lengths)):
    if seqlen <= 9:
        seq_tensor[idx, :seqlen] = LongTensor(seq)
    else:
        seq_tensor[idx, :seqlen] = LongTensor(seq[:9])
print(seq_tensor)
seq_tensor.shape
embedded_seq_tensor = embed(seq_tensor)
print(embedded_seq_tensor)
print(embedded_seq_tensor.shape)
target = sorted(set(all_categories))
vectorized_y = [target.index(tok) for tok in DF[1]]
print(DF[1][:5])
print(vectorized_y[:5])
target_tensor = torch.tensor(vectorized_y,dtype = torch.long)
print(target_tensor.shape)

['Abl', 'Adsit', 'Ajdrna', 'Alt', 'Antonowitsch']
[[5, 32, 42], [5, 34, 49, 39, 50], [5, 40, 34, 48, 44, 31], [5, 42, 50], [5, 44, 50, 45, 44, 45, 53, 39, 50, 49, 33, 38]]
tensor([3, 5, 6,  ..., 8, 7, 5])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
tensor([[ 5, 32, 42,  ...,  0,  0,  0],
        [ 5, 34, 49,  ...,  0,  0,  0],
        [ 5, 40, 34,  ...,  0,  0,  0],
        ...,
        [30, 39, 35,  ..., 31, 41,  0],
        [30, 39, 35,  ..., 41,  0,  0],
        [30, 51, 48,  ...,  0,  0,  0]])
tensor([[[-0.1936, -1.2994, -0.4664,  ..., -0.3159, -1.3663, -0.5370],
         [-0.4547,  0.2550,  1.6509,  ..., -0.7490, -1.1880,  1.0071],
         [-0.3689, -2.0978,  0.3502,  ..., -0.6274,  0.1800, -0.4970],
         ...,
         [-0.6283,  1.4404,  1.8816,  ...,  0.0653, -0.3580, -0.2610],
         [-0.6283,  1.440

In [18]:
print('======= 2.1(b) Accuracy Report After Chosen Epochs =======')
kfold_cv = KFold(n_splits=5, random_state=None, shuffle=True)
rnn = RNN_Batch()
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
loss_func = nn.CrossEntropyLoss()
n_epoch = 1000 
for epoch in range(n_epoch):
    accuracies_test = []
    accuracies_train = []
    for train_index, test_index in kfold_cv.split(np.arange(0, total_samples)):
        X_train, X_test, y_train, y_test = embedded_seq_tensor[train_index], embedded_seq_tensor[test_index], target_tensor[train_index], target_tensor[test_index]
        optimizer.zero_grad()
        output = rnn(X_train)
        loss = loss_func(output, y_train) 
        loss.backward(retain_graph=True)
        optimizer.step()
        test_output = rnn(X_test)
        train_output = rnn(X_train)
        pred_test = torch.max(test_output, 1)[1]
        pred_train = torch.max(train_output, 1)[1]
        accuracies_test.append(accuracy_score(y_test, pred_test))
        accuracies_train.append(accuracy_score(y_train, pred_train))
    print("Epoch: ", epoch, "| Train Loss: %.4f" % loss.item(), '| Test accuracy: %f' % np.mean(accuracies_test))
    print("Epoch: ", epoch, "| Train Loss: %.4f" % loss.item(), '| Train accuracy: %f' % np.mean(accuracies_train))

======= 2.1(b) Accuracy Report After Chosen Epochs =======
Epoch:  0 | Train Loss: 1.7909 | Test accuracy: 0.449289
Epoch:  0 | Train Loss: 1.7909 | Train accuracy: 0.449412
Epoch:  1 | Train Loss: 1.6684 | Test accuracy: 0.495866
Epoch:  1 | Train Loss: 1.6684 | Train accuracy: 0.496625
Epoch:  2 | Train Loss: 1.6069 | Test accuracy: 0.511160
Epoch:  2 | Train Loss: 1.6069 | Train accuracy: 0.512653
Epoch:  3 | Train Loss: 1.5177 | Test accuracy: 0.527447
Epoch:  3 | Train Loss: 1.5177 | Train accuracy: 0.525780
Epoch:  4 | Train Loss: 1.5052 | Test accuracy: 0.537412
Epoch:  4 | Train Loss: 1.5052 | Train accuracy: 0.538495
Epoch:  5 | Train Loss: 1.4708 | Test accuracy: 0.560527
Epoch:  5 | Train Loss: 1.4708 | Train accuracy: 0.561298
Epoch:  6 | Train Loss: 1.4137 | Test accuracy: 0.583592
Epoch:  6 | Train Loss: 1.4137 | Train accuracy: 0.582818
Epoch:  7 | Train Loss: 1.3426 | Test accuracy: 0.608449
Epoch:  7 | Train Loss: 1.3426 | Train accuracy: 0.608685
Epoch:  8 | Train Los

Epoch:  70 | Train Loss: 0.3534 | Test accuracy: 0.890405
Epoch:  70 | Train Loss: 0.3534 | Train accuracy: 0.890592
Epoch:  71 | Train Loss: 0.3450 | Test accuracy: 0.893644
Epoch:  71 | Train Loss: 0.3450 | Train accuracy: 0.893843
Epoch:  72 | Train Loss: 0.3358 | Test accuracy: 0.897430
Epoch:  72 | Train Loss: 0.3358 | Train accuracy: 0.896383
Epoch:  73 | Train Loss: 0.3290 | Test accuracy: 0.899074
Epoch:  73 | Train Loss: 0.3290 | Train accuracy: 0.898974
Epoch:  74 | Train Loss: 0.3235 | Test accuracy: 0.901564
Epoch:  74 | Train Loss: 0.3235 | Train accuracy: 0.901340
Epoch:  75 | Train Loss: 0.3187 | Test accuracy: 0.903408
Epoch:  75 | Train Loss: 0.3187 | Train accuracy: 0.904067
Epoch:  76 | Train Loss: 0.3023 | Test accuracy: 0.906695
Epoch:  76 | Train Loss: 0.3023 | Train accuracy: 0.906633
Epoch:  77 | Train Loss: 0.3011 | Test accuracy: 0.908987
Epoch:  77 | Train Loss: 0.3011 | Train accuracy: 0.909348
Epoch:  78 | Train Loss: 0.2890 | Test accuracy: 0.911477
Epoch:

Epoch:  140 | Train Loss: 0.1422 | Test accuracy: 0.954618
Epoch:  140 | Train Loss: 0.1422 | Train accuracy: 0.955726
Epoch:  141 | Train Loss: 0.1365 | Test accuracy: 0.957258
Epoch:  141 | Train Loss: 0.1365 | Train accuracy: 0.958117
Epoch:  142 | Train Loss: 0.1335 | Test accuracy: 0.960148
Epoch:  142 | Train Loss: 0.1335 | Train accuracy: 0.959736
Epoch:  143 | Train Loss: 0.1263 | Test accuracy: 0.960646
Epoch:  143 | Train Loss: 0.1263 | Train accuracy: 0.960646
Epoch:  144 | Train Loss: 0.1230 | Test accuracy: 0.961592
Epoch:  144 | Train Loss: 0.1230 | Train accuracy: 0.961679
Epoch:  145 | Train Loss: 0.1199 | Test accuracy: 0.961841
Epoch:  145 | Train Loss: 0.1199 | Train accuracy: 0.962763
Epoch:  146 | Train Loss: 0.1170 | Test accuracy: 0.962639
Epoch:  146 | Train Loss: 0.1170 | Train accuracy: 0.963236
Epoch:  147 | Train Loss: 0.1128 | Test accuracy: 0.963037
Epoch:  147 | Train Loss: 0.1128 | Train accuracy: 0.963909
Epoch:  148 | Train Loss: 0.1121 | Test accuracy

Epoch:  209 | Train Loss: 0.0636 | Test accuracy: 0.969812
Epoch:  209 | Train Loss: 0.0636 | Train accuracy: 0.970808
Epoch:  210 | Train Loss: 0.0612 | Test accuracy: 0.969413
Epoch:  210 | Train Loss: 0.0612 | Train accuracy: 0.970933
Epoch:  211 | Train Loss: 0.0638 | Test accuracy: 0.968367
Epoch:  211 | Train Loss: 0.0638 | Train accuracy: 0.971107
Epoch:  212 | Train Loss: 0.0634 | Test accuracy: 0.969463
Epoch:  212 | Train Loss: 0.0634 | Train accuracy: 0.970920
Epoch:  213 | Train Loss: 0.0607 | Test accuracy: 0.969812
Epoch:  213 | Train Loss: 0.0607 | Train accuracy: 0.970895
Epoch:  214 | Train Loss: 0.0620 | Test accuracy: 0.969563
Epoch:  214 | Train Loss: 0.0620 | Train accuracy: 0.970908
Epoch:  215 | Train Loss: 0.0625 | Test accuracy: 0.969812
Epoch:  215 | Train Loss: 0.0625 | Train accuracy: 0.970870
Epoch:  216 | Train Loss: 0.0600 | Test accuracy: 0.969812
Epoch:  216 | Train Loss: 0.0600 | Train accuracy: 0.970820
Epoch:  217 | Train Loss: 0.0631 | Test accuracy

Epoch:  278 | Train Loss: 0.0554 | Test accuracy: 0.969164
Epoch:  278 | Train Loss: 0.0554 | Train accuracy: 0.971294
Epoch:  279 | Train Loss: 0.0544 | Test accuracy: 0.969662
Epoch:  279 | Train Loss: 0.0544 | Train accuracy: 0.971157
Epoch:  280 | Train Loss: 0.0535 | Test accuracy: 0.970111
Epoch:  280 | Train Loss: 0.0535 | Train accuracy: 0.971107
Epoch:  281 | Train Loss: 0.0520 | Test accuracy: 0.970061
Epoch:  281 | Train Loss: 0.0520 | Train accuracy: 0.971157
Epoch:  282 | Train Loss: 0.0523 | Test accuracy: 0.968118
Epoch:  282 | Train Loss: 0.0523 | Train accuracy: 0.971518
Epoch:  283 | Train Loss: 0.0537 | Test accuracy: 0.968268
Epoch:  283 | Train Loss: 0.0537 | Train accuracy: 0.971580
Epoch:  284 | Train Loss: 0.0521 | Test accuracy: 0.969413
Epoch:  284 | Train Loss: 0.0521 | Train accuracy: 0.971281
Epoch:  285 | Train Loss: 0.0535 | Test accuracy: 0.969712
Epoch:  285 | Train Loss: 0.0535 | Train accuracy: 0.971219
Epoch:  286 | Train Loss: 0.0530 | Test accuracy

Epoch:  347 | Train Loss: 0.0514 | Test accuracy: 0.968566
Epoch:  347 | Train Loss: 0.0514 | Train accuracy: 0.971555
Epoch:  348 | Train Loss: 0.0522 | Test accuracy: 0.969712
Epoch:  348 | Train Loss: 0.0522 | Train accuracy: 0.971244
Epoch:  349 | Train Loss: 0.0497 | Test accuracy: 0.968965
Epoch:  349 | Train Loss: 0.0497 | Train accuracy: 0.971468
Epoch:  350 | Train Loss: 0.0516 | Test accuracy: 0.969712
Epoch:  350 | Train Loss: 0.0516 | Train accuracy: 0.971219
Epoch:  351 | Train Loss: 0.0515 | Test accuracy: 0.968417
Epoch:  351 | Train Loss: 0.0515 | Train accuracy: 0.971593
Epoch:  352 | Train Loss: 0.0514 | Test accuracy: 0.969613
Epoch:  352 | Train Loss: 0.0514 | Train accuracy: 0.971207
Epoch:  353 | Train Loss: 0.0516 | Test accuracy: 0.969264
Epoch:  353 | Train Loss: 0.0516 | Train accuracy: 0.971356
Epoch:  354 | Train Loss: 0.0494 | Test accuracy: 0.969214
Epoch:  354 | Train Loss: 0.0494 | Train accuracy: 0.971381
Epoch:  355 | Train Loss: 0.0516 | Test accuracy

KeyboardInterrupt: 

In [ ]:
print('======= 2.2(a) Accuracy Report After Five Epochs =======')
kfold_cv = KFold(n_splits=5, random_state=None, shuffle=True)
lstm = LSTM()
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
loss_func = nn.CrossEntropyLoss()
n_epoch = 5 
for epoch in range(n_epoch):
    accuracies_test = []
    accuracies_train = []
    for train_index, test_index in kfold_cv.split(np.arange(0, total_samples)):
        X_train, X_test, y_train, y_test = embedded_seq_tensor[train_index], embedded_seq_tensor[test_index], target_tensor[train_index], target_tensor[test_index]
        optimizer.zero_grad()
        output = lstm(X_train)
        loss = loss_func(output, y_train) 
        loss.backward(retain_graph=True)
        optimizer.step()
        test_output = lstm(X_test)
        train_output = lstm(X_train)
        pred_test = torch.max(test_output, 1)[1]
        pred_train = torch.max(train_output, 1)[1]
        accuracies_test.append(accuracy_score(y_test, pred_test))
        accuracies_train.append(accuracy_score(y_train, pred_train))
    print("Epoch: ", epoch, "| Train Loss: %.4f" % loss.item(), '| Test accuracy: %f' % np.mean(accuracies_test))
    print("Epoch: ", epoch, "| Train Loss: %.4f" % loss.item(), '| Train accuracy: %f' % np.mean(accuracies_train))

In [20]:
print('======= 2.2(b) Accuracy Report After Chosen Epochs =======')
kfold_cv = KFold(n_splits=5, random_state=None, shuffle=True)
lstm = LSTM()
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
loss_func = nn.CrossEntropyLoss()
n_epoch = 1000 
for epoch in range(n_epoch):
    accuracies_test = []
    accuracies_train = []
    for train_index, test_index in kfold_cv.split(np.arange(0, total_samples)):
        X_train, X_test, y_train, y_test = embedded_seq_tensor[train_index], embedded_seq_tensor[test_index], target_tensor[train_index], target_tensor[test_index]
        optimizer.zero_grad()
        output = lstm(X_train)
        loss = loss_func(output, y_train) 
        loss.backward(retain_graph=True)
        optimizer.step()
        test_output = lstm(X_test)
        train_output = lstm(X_train)
        pred_test = torch.max(test_output, 1)[1]
        pred_train = torch.max(train_output, 1)[1]
        accuracies_test.append(accuracy_score(y_test, pred_test))
        accuracies_train.append(accuracy_score(y_train, pred_train))
    print("Epoch: ", epoch, "| Train Loss: %.4f" % loss.item(), '| Test accuracy: %f' % np.mean(accuracies_test))
    print("Epoch: ", epoch, "| Train Loss: %.4f" % loss.item(), '| Train accuracy: %f' % np.mean(accuracies_train))

======= 2.2(b) Accuracy Report After Chosen Epochs =======
Epoch:  0 | Train Loss: 1.9653 | Test accuracy: 0.459151
Epoch:  0 | Train Loss: 1.9653 | Train accuracy: 0.459761
Epoch:  1 | Train Loss: 1.6639 | Test accuracy: 0.511010
Epoch:  1 | Train Loss: 1.6639 | Train accuracy: 0.511184
Epoch:  2 | Train Loss: 1.5429 | Test accuracy: 0.530537
Epoch:  2 | Train Loss: 1.5429 | Train accuracy: 0.529914
Epoch:  3 | Train Loss: 1.4851 | Test accuracy: 0.564015
Epoch:  3 | Train Loss: 1.4851 | Train accuracy: 0.562718
Epoch:  4 | Train Loss: 1.3936 | Test accuracy: 0.598536
Epoch:  4 | Train Loss: 1.3936 | Train accuracy: 0.599121
Epoch:  5 | Train Loss: 1.3171 | Test accuracy: 0.622796
Epoch:  5 | Train Loss: 1.3171 | Train accuracy: 0.622198
Epoch:  6 | Train Loss: 1.2565 | Test accuracy: 0.635051
Epoch:  6 | Train Loss: 1.2565 | Train accuracy: 0.634951
Epoch:  7 | Train Loss: 1.1906 | Test accuracy: 0.652387
Epoch:  7 | Train Loss: 1.1906 | Train accuracy: 0.653245
Epoch:  8 | Train Los

Epoch:  70 | Train Loss: 0.1438 | Test accuracy: 0.954219
Epoch:  70 | Train Loss: 0.1438 | Train accuracy: 0.955266
Epoch:  71 | Train Loss: 0.1397 | Test accuracy: 0.956063
Epoch:  71 | Train Loss: 0.1397 | Train accuracy: 0.956175
Epoch:  72 | Train Loss: 0.1381 | Test accuracy: 0.956810
Epoch:  72 | Train Loss: 0.1381 | Train accuracy: 0.957644
Epoch:  73 | Train Loss: 0.1316 | Test accuracy: 0.957756
Epoch:  73 | Train Loss: 0.1316 | Train accuracy: 0.958416
Epoch:  74 | Train Loss: 0.1280 | Test accuracy: 0.959301
Epoch:  74 | Train Loss: 0.1280 | Train accuracy: 0.959537
Epoch:  75 | Train Loss: 0.1193 | Test accuracy: 0.960546
Epoch:  75 | Train Loss: 0.1193 | Train accuracy: 0.960658
Epoch:  76 | Train Loss: 0.1184 | Test accuracy: 0.961293
Epoch:  76 | Train Loss: 0.1184 | Train accuracy: 0.961555
Epoch:  77 | Train Loss: 0.1145 | Test accuracy: 0.963136
Epoch:  77 | Train Loss: 0.1145 | Train accuracy: 0.962663
Epoch:  78 | Train Loss: 0.1112 | Test accuracy: 0.963186
Epoch:

KeyboardInterrupt: 

In [21]:
print('======= 2.3 Accuracy Report =======')
skfold_cv = StratifiedKFold(n_splits=5)
lstm = LSTM()
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
loss_func = nn.CrossEntropyLoss()
n_epoch = 20 
for epoch in range(n_epoch):
    accuracies_test = []
    accuracies_train = []
    for Train, Test in skfold_cv.split(embedded_seq_tensor,target_tensor):
        optimizer.zero_grad()
        output = lstm(embedded_seq_tensor[Train])
        loss = loss_func(output, target_tensor[Train]) 
        loss.backward(retain_graph=True)
        optimizer.step()
        test_output = lstm(embedded_seq_tensor[Test])
        train_output = lstm(embedded_seq_tensor[Train])
        pred_test = torch.max(test_output, 1)[1]
        pred_train = torch.max(train_output, 1)[1]
        accuracy_test = accuracy_score(target_tensor[Test], pred_test)
        accuracy_train = accuracy_score(target_tensor[Train], pred_train)
        accuracies_test.append(accuracy_test)
        accuracies_train.append(accuracy_train)
    print("Epoch: ", epoch, "| Train Loss: %.4f" % loss.item(), '| Test accuracy: %f' % np.mean(accuracies_test))
    print("Epoch: ", epoch, "| Train Loss: %.4f" % loss.item(), '| Train accuracy: %f' % np.mean(accuracies_train))

======= 2.3 Accuracy Report =======
Epoch:  0 | Train Loss: 1.9788 | Test accuracy: 0.463892
Epoch:  0 | Train Loss: 1.9788 | Train accuracy: 0.467744
Epoch:  1 | Train Loss: 1.6359 | Test accuracy: 0.512963
Epoch:  1 | Train Loss: 1.6359 | Train accuracy: 0.514356
Epoch:  2 | Train Loss: 1.5414 | Test accuracy: 0.543095
Epoch:  2 | Train Loss: 1.5414 | Train accuracy: 0.542666
Epoch:  3 | Train Loss: 1.4581 | Test accuracy: 0.557299
Epoch:  3 | Train Loss: 1.4581 | Train accuracy: 0.559540
Epoch:  4 | Train Loss: 1.3828 | Test accuracy: 0.606119
Epoch:  4 | Train Loss: 1.3828 | Train accuracy: 0.604313
Epoch:  5 | Train Loss: 1.3203 | Test accuracy: 0.618276
Epoch:  5 | Train Loss: 1.3203 | Train accuracy: 0.618474
Epoch:  6 | Train Loss: 1.2545 | Test accuracy: 0.633217
Epoch:  6 | Train Loss: 1.2545 | Train accuracy: 0.633082
Epoch:  7 | Train Loss: 1.1908 | Test accuracy: 0.646874
Epoch:  7 | Train Loss: 1.1908 | Train accuracy: 0.649471
Epoch:  8 | Train Loss: 1.1237 | Test accura